In [ ]:

from google.colab import drive
import torch
import pickle
import os

drive.mount('/content/drive', force_remount=True)
SAVE_PATH = "/content/drive/MyDrive/KD_Project"
os.makedirs(SAVE_PATH, exist_ok=True)

Mounted at /content/drive


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("glue", "sst2")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(batch["sentence"], truncation=True, padding="max_length", max_length=128)

tokenized_ds = dataset.map(tokenize_fn, batched=True)
tokenized_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

args = TrainingArguments(
    output_dir=f"{SAVE_PATH}/bert_checkpoints",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"]
)

trainer.train()
model.save_pretrained(f"{SAVE_PATH}/final_teacher")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 No


wandb: WARNING Invalid choice
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.302000
1000,0.206500
1500,0.183600
2000,0.164300
2500,0.117700
3000,0.104200
3500,0.103500
4000,0.101700


In [7]:
from torch.utils.data import DataLoader
import pickle

# 1. Create a "Conveyor Belt" for the data
train_loader = DataLoader(tokenized_ds["train"], batch_size=32, shuffle=False)

model.eval() # 2. Set model to "Listen" mode (not learning mode)
all_teacher_logits = []

with torch.no_grad(): # 3. Don't waste memory calculating gradients
    for batch in train_loader:
        inputs = {'input_ids': batch['input_ids'].to(model.device),
                  'attention_mask': batch['attention_mask'].to(model.device)}
        outputs = model(**inputs) # 4. Get the Professor's raw opinion
        all_teacher_logits.append(outputs.logits.cpu())

# 5. Save the opinions to a file
teacher_logits_tensor = torch.cat(all_teacher_logits, dim=0)
with open(f"{SAVE_PATH}/teacher_logits.pkl", "wb") as f:
    pickle.dump(teacher_logits_tensor, f)